In [1]:
import os
import torch
import numpy as np
import skimage.transform
import matplotlib.pyplot as plt
from easydict import EasyDict as edict

from main_monodepth_pytorch import Model
%reload_ext autoreload
%autoreload 2

## Train

Check if CUDA is available

In [2]:
torch.cuda.is_available()

True

In [3]:
torch.__version__

'0.4.1'

In [4]:
torch.cuda.empty_cache()

In [7]:
dict_parameters = edict({'data_dir':'../KITTI/one_example/',
                         'val_data_dir':'../KITTI/one_example/',
                         'model_path':'../KITTI/models/test_model.pth',
                         'output_directory':'../KITTI/models/',
                         'input_height':256,
                         'input_width':512,
                         'model':'resnet50',
                         'pretrained':False,
                         'mode':'train',
                         'epochs':500,
                         'learning_rate':1e-1,
                         'batch_size':2,
                         'adjust_lr':True,
                         'device':'cuda:0',
                         'do_augmentation':True,
                         'augment_parameters':[0.8, 1.2, 0.5, 2.0, 0.8, 1.2],
                         'print_images':False,
                         'print_weights':False,
                         'input_channels':3})

In [8]:
model = Model(dict_parameters)
# model.load('data/models/model_to_load.pth')

Use a dataset with 1 images
Use a dataset with 1 images


In [9]:
model.train()

Val_loss: 7.089230060577393
Epoch: 1 train_loss: 7.575516700744629 val_loss: nan time: 0.306 s
Epoch: 2 train_loss: 7.23919677734375 val_loss: 8.753206253051758 time: 0.366 s
Model_saved
Epoch: 3 train_loss: 7.101390361785889 val_loss: 8.742904663085938 time: 0.34 s
Model_saved
Epoch: 4 train_loss: 6.942844390869141 val_loss: 9.980751037597656 time: 0.311 s
Epoch: 5 train_loss: 7.12115478515625 val_loss: 8.9718017578125 time: 0.283 s
Epoch: 6 train_loss: 6.444443702697754 val_loss: 10.061543464660645 time: 0.333 s
Epoch: 7 train_loss: 6.951641082763672 val_loss: 8.723533630371094 time: 0.426 s
Model_saved
Epoch: 8 train_loss: 6.507801055908203 val_loss: 10.202131271362305 time: 0.313 s
Epoch: 9 train_loss: 6.406336307525635 val_loss: 10.135198593139648 time: 0.282 s
Epoch: 10 train_loss: 6.371866703033447 val_loss: 10.093294143676758 time: 0.404 s
Epoch: 11 train_loss: 6.658284664154053 val_loss: 9.929451942443848 time: 0.285 s
Epoch: 12 train_loss: 6.1803507804870605 val_loss: 10.0518

KeyboardInterrupt: 

## Test the model

In [ ]:
dict_parameters_test = edict({'data_dir':'../KITTI/one_example/',
                              'model_path':'../KITTI/models/test_model.pth',
                              'output_directory':'../KITTI/models/',
                              'input_height':256,
                              'input_width':512,
                              'model':'resnet18_md',
                              'mode':'test',
                              'device':'cuda:0'})  # torch.FloatTensor' for cpu mode
model_test = Model(dict_parameters_test)

In [ ]:
model_test.test()

In [ ]:
disp = np.load('../KITTI/models/disparities_pp.npy')  # Or disparities_pp.npy for postprocessed

In [ ]:
disp_to_img = skimage.transform.resize(disp[0].squeeze(), [375, 1242], mode='constant')
plt.imshow(disp_to_img, cmap='plasma')

Save a color image

In [ ]:
plt.imsave(os.path.join(dict_parameters_test.output_directory,
                        dict_parameters_test.model_path.split('/')[-1][:-4]+'_test0.png'), disp_to_img, cmap='plasma')

Save a grayscale image

In [ ]:
plt.imsave(os.path.join(dict_parameters_test.output_directory,
                        dict_parameters_test.model_path.split('/')[-1][:-4]+'gray.png'), disp_to_img, cmap='gray')